In [1]:
import numpy as np

In [2]:
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
import matplotlib.patches as patches
def plot_rectangle(rec, ax=None):
    if ax is None: 
        ax = plt.subplot(aspect='equal')
    
    if isinstance(rec, (list, tuple)):
        for r in rec: 
            plot_rectangle(r,ax)
    
    else:
        size = (rec.maxes-rec.mins)
        ax.add_patch(patches.Rectangle(rec.mins, size[0], size[1], fill=False, zorder=-1))
    
    

In [4]:
#diff = np.float32(0.033068776)
diff = np.float32(0.03306878)
global_min = -31*diff
global_max = 31*diff

dom_maxs = np.array([global_max]*3, dtype=np.float64)
dom_mins = np.array([global_min]*3, dtype=np.float64)
tau = 0.2/12600 # 0.2 times mean interparticle separation
#buffer_tau = diff*5./150.
Ngrid = 62

In [5]:
import sparkhpc

In [6]:
ncores = 8

In [7]:
import os
os.environ['SPARK_HOME'] = '/home/ics/roskar/spark'
os.environ['SPARK_DRIVER_MEMORY'] = '16G'

In [8]:
import findspark
findspark.init()

In [9]:
import sparkhpc

In [10]:
clusterid=None

if clusterid is None: 
    sj = sparkhpc.sparkjob.sparkjob(ncores=ncores,
                                memory=50000,
                                walltime='08:00', 
                                cores_per_executor=4, 
                                memory_per_executor=50000)
    sj.wait_to_start()
else:
    sj = sparkhpc.sparkjob.sparkjob(clusterid=clusterid)

INFO:sparkhpc.sparkjob:Submitted cluster 0


In [23]:
sc = sparkhpc.start_spark(master=sj.master_url, spark_conf='../conf', 
                          profiling=True, executor_memory='20000M', graphframes_package='graphframes:graphframes:0.3.0-spark2.0-s_2.11')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=spark-fof, master=spark://x10y06:7077) created by __init__ at /home/ics/roskar/src/sparkhpc/sparkhpc/__init__.py:63 

In [24]:
sc.setCheckpointDir('file:///zbox/data/roskar/work/checkpoint')

In [25]:
import spark_fof
reload(spark_fof.spark_fof)
reload(spark_fof)
path = '/zbox/trove/euclid/2Tlc-final/'

nMinMembers = 8
nBins = 62
minblock = 30
maxblock = 34
fof_analyzer = spark_fof.spark_fof.LCFOFAnalyzer(sc, path, nMinMembers, nBins, tau, dom_mins, dom_maxs, blockids=range(minblock,maxblock), buffer_tau=tau*2)

spark_fof: Number of input files:  64
spark_fof: Total number of particles:  4695434296


In [14]:
# sub-sample
#fof_analyzer.particle_rdd = fof_analyzer.particle_rdd.map(lambda arr: np.random.choice(arr, np.int(np.ceil(len(arr)*0.2))), preservesPartitioning=True).cache()

In [26]:
fof_analyzer.particle_rdd.setName('particle_rdd')

particle_rdd PythonRDD[432] at RDD at PythonRDD.scala:48

In [ ]:
%time fof_analyzer.finalize_groups()

In [ ]:
!scancel -u roskar